Poisson Equation
===

Solving the Poisson Equation
==

$$
-\Delta u(x) = f(x) \quad \forall \, x \in \Omega
$$

The right hand side $f$ is a given function, and we search for the solution $u$. The domain $\Omega$ is a subset of $R^n$. The Poisson equation is a model for many physical phenomena:
* f can be a heat source distribution, and u is the temperature
* f can be an electric charge distribution, and u is the electrostatic potential

To select a unique solution $u$ we have to specify boundary conditions, for example homogeneous Dirichlet boundary conditions

$$
u(x) = 0 \quad \forall \, x \in \partial \Omega
$$

Weak formulation
---

$$
- \int_\Omega \Delta u(x) v(x) dx = \int_\Omega f(x) v(x) dx
$$


$$
\int_\Omega \nabla u \nabla v - \int_{\partial \Omega} \frac{\partial u}{\partial n} v = \int_\Omega f v
$$

## Dirichlet

In the case of Dirichlet boundary conditions we allow only test-functions $v$ such that $v(x) = 0$ on the boundary $\partial \Omega$.

The weak form then becomes: 

$$
\int_\Omega \nabla u \nabla v = \int_\Omega f v 
$$

Or using linear and bilinear forms

$$
\text{find} \, u \in H_0^1 : \quad A(u,v) = f(v) \quad \forall \, v \in H_0^1
$$

### Data
For our problem, $$ u = 16x(1-x)y(1-y) $$ therefore $$ \nabla u = (16(1-2x)y(1-y), 16x(1-x)(1-2y)) $$
and $$ f = 32 (y(1-y)+x(1-x)) $$

Import Netgen/NGSolve Python modules:

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
# unit_square is the predefined domain (0,1)^2
from netgen.occ import unit_square


# old
# from netgen.geom2d import unit_square
# import netgen.gui
# %gui tk

In [ ]:
DrawGeo(unit_square.shape)

In [ ]:
# Generate mesh for different h for unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw(mesh)

In [ ]:
exact = 16*x*(1-x)*y*(1-y)
grad_exact = CoefficientFunction( (16*(1-2*x)*y*(1-y), 16*x*(1-x)*(1-2*y)) )
source = 32 * (y*(1-y)+x*(1-x))

In [ ]:
Draw(exact, mesh, "exact")
Draw(grad_exact, mesh, "grad_exact")

In [ ]:
# Define Finite Element Space
fes = H1(mesh, order=3, dirichlet="bottom|right|top|left")

# Declare test, trial and grid functions
u, v = fes.TnT()

# Define and assemble forms
f = LinearForm(fes)
f += source * v * dx
f.Assemble()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
a.Assemble()

# Solve the linear system
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
grad_gfu = grad(gfu)


In [ ]:
Draw(gfu, mesh, "gfu")
Draw(grad_gfu, mesh, "grad_gfu")

## Error Analysis and plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dofs = []
errors = []
grad_errors = []

mesh_size = [0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625]
for h in mesh_size:
    # Generate mesh for different h for unit_square
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))

    # Define Finite Element Space 
    fes = H1(mesh, order=1, dirichlet="bottom|right|top|left")
    dofs.append(fes.ndof)

    # Declare test, trial and grid functions
    u, v = fes.TnT()

    # Define and assemble forms
    f = LinearForm(fes)
    f += source * v * dx
    f.Assemble()

    a = BilinearForm(fes)
    a += grad(u)*grad(v)*dx
    a.Assemble()

    # Solve the linear system
    gfu = GridFunction(fes)
    gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
    grad_gfu = grad(gfu)

    # Compute error
    err = sqrt (Integrate ( (gfu-exact)*(gfu-exact), mesh))
    grad_err = sqrt (Integrate ( (grad_gfu-grad_exact)*(grad_gfu-grad_exact), mesh))
    print ("h = ", h, "L2-error:", err, "grad-error:", grad_err)

    errors.append(err)
    grad_errors.append(grad_err)

    

In [ ]:
print("dofs: ", dofs)
print("errors: ", errors)
print("grad_errors: ", grad_errors)

In [ ]:
for i in range(len(mesh_size)-1):
    print("Convergence rate:", np.log(errors[i+1]/errors[i])/np.log(mesh_size[i+1]/mesh_size[i]))
    print("Convergence rate for grad errors:", np.log(grad_errors[i+1]/grad_errors[i])/np.log(mesh_size[i+1]/mesh_size[i]))

In [ ]:

plt.loglog(dofs, np.divide(np.ones(len(dofs)), np.array(dofs)), label="Reference Line")
plt.loglog(dofs,errors, "-o", label="L2 Error")
plt.title("Error Estimation Plot")
plt.xlabel("Mesh Size")
plt.ylabel("L2 Error")
plt.legend()
plt.show()